<div id = 3 style="color:white;
           display:fill;
           border-radius:20px;
           background-color:#b02222;
           font-size:100%;
           font-family:Verdana;
           letter-spacing:0.5px;
           display:flex;
            justify-content:center;">

<h2 style="padding: 6rem;
              color:white;
          text-align:center;
          margin:10 auto;
          ">
    ITERATION 10: Identify, Isolate and Utilize more popular Key Features


</h2>
</div>

In [1]:
import json

with open('../z_envs/_envs.json') as f:
    env_vars = json.loads(f.read())
print(env_vars)

run_env = env_vars['notebook_environment']

if run_env not in ['colab', 'gradient', 'cloud']:
    cloud_run = False
else:
    cloud_run = True
    import sys
    import os

    module_path = os.path.abspath(os.path.join('..', '..'))
    if module_path not in sys.path:
        sys.path.append(module_path)
        
    %pip install missingno

from functions_0__common_20221116 import get_columns
from functions_b__get_the_data_20221116 import set_csv_directory, get_combined_dataset
from functions_d1__prepare_cleanse_data_20221116 import add_supplements, tidy_dataset
from functions_d2__transform_enrich_data_20221116 import feature_engineer

{'notebook_environment': 'local', 'use_gpu': False, 'debug_mode': False, 'quick_mode': False, 'quick_override_cv_splits': 2, 'quick_override_n_iter': 10, 'quick_override_n_jobs': 3}


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored

import warnings

In [3]:
VERSION = '10'

warnings.filterwarnings("ignore")

In [4]:
LABEL = 'Price'

columns, booleans, floats, categories, custom, wildcard = get_columns(version=VERSION)

print(colored(f"features", "blue"), "-> ", columns)
print(colored(f"+ ", "blue"), "-> ", wildcard)

columns.insert(0, LABEL)
original_columns = columns
print(colored(f"label", "green"), "-> ", LABEL)

features ->  ['bedrooms', 'bathrooms', 'nearestStation', 'location.latitude', 'location.longitude', 'latitude_deviation', 'longitude_deviation', 'tenure.tenureType']
+  ->  ['feature__']
label ->  Price


In [7]:
IN_COLAB = False

filename = f'df_listings_v{VERSION}.csv'
remote_pathname = f'https://raw.githubusercontent.com/jayportfolio/capstone_streamlit/main/data/final/{filename}'

df_pathname_raw = f'../../data/source/{filename}'
#df_pathname_tidy = f'../../data/final/{filename}'

df_pathname = df_pathname_raw if not cloud_run else remote_pathname
print(df_pathname)
df_orig = pd.read_csv(df_pathname, on_bad_lines='error', index_col=0)

../../data/source/df_listings_v10.csv


In [8]:
df_prep = add_supplements(df_orig, version=VERSION)
df_prep = feature_engineer(df_prep, version=VERSION)

['\nground rent: £150 p.a. ' '   semi-detached property'
 " 'for sale by online auction, starting bid of£250,000, terms and conditions apply'."
 ...
 '‘a’rated energy efficiency apartments ensuring huge savings on energy bills'
 '‘mansion block’ style ex-local authority apartment' '‘nash built’ house'] [1 1 1 ... 1 1 1]
118449 118449


In [9]:
updated_columns = False
columns = original_columns

for each in df_prep.columns:
    for prefix in wildcard:
        if each.startswith(prefix):
            columns.append(each)
            updated_columns = True

if updated_columns:
    print(colored(f"columns updated to:", "yellow"), "-> ", columns)
df_prep

columns updated to: ->  ['Price', 'bedrooms', 'bathrooms', 'nearestStation', 'location.latitude', 'location.longitude', 'latitude_deviation', 'longitude_deviation', 'tenure.tenureType', 'feature__1 bedroom', 'feature__2 bedrooms', 'feature__2 double bedrooms', 'feature__allocated parking', 'feature__allocated parking space', 'feature__balcony', 'feature__bathroom', 'feature__chain free', 'feature__close to local amenities', 'feature__communal garden', 'feature__communal gardens', 'feature__double bedroom', 'feature__double glazed', 'feature__double glazing', 'feature__epc rating c', 'feature__epc rating d', 'feature__excellent location', 'feature__excellent transport links', 'feature__family bathroom', 'feature__first floor', 'feature__fitted kitchen', 'feature__garage', 'feature__garden', 'feature__gas central heating', 'feature__great location', 'feature__ground floor', 'feature__kitchen', 'feature__leasehold', 'feature__long lease', 'feature__modern bathroom', 'feature__modern kitch

,Links,Price,version,borough,referencing_link,add_info,date_scraped,version_listing,link,distance_to_any_train,...,feature__reception room,feature__separate kitchen,feature__share of freehold,feature__three bedrooms,feature__three double bedrooms,feature__top floor,feature__two bathrooms,feature__two bedrooms,feature__two double bedrooms,feature__two reception rooms
14520525,https://www.rightmove.co.uk/properties/1452052...,550000,v0021_300622,NaN,https://www.rightmove.co.uk/property-for-sale/...,"Added on 24/05/2022 by Arlington Estates, London",2022-07-06 19:13:02,v0021_300622,https://www.rightmove.co.uk/properties/14520525,0.3,...,0,1,0,0,0,0,0,0,0,0
27953107,https://www.rightmove.co.uk/properties/2795310...,400000,v0021_300622,NaN,https://www.rightmove.co.uk/property-for-sale/...,"Added on 16/06/2022 by Lawlors Estate Agents, ...",2022-08-14 10:33:29,v0021_300622,https://www.rightmove.co.uk/properties/27953107,0.3,...,0,0,0,0,0,1,0,0,1,0
33593487,https://www.rightmove.co.uk/properties/3359348...,579950,v0021_300622,NaN,https://www.rightmove.co.uk/property-for-sale/...,"Added on 11/07/2022 by Websters Estate Agents,...",2022-07-16 13:58:28,v0021_300622,https://www.rightmove.co.uk/properties/33593487,0.4,...,0,0,0,0,0,0,0,0,0,0
35271294,https://www.rightmove.co.uk/properties/3527129...,370000,v0021_300622,NaN,https://www.rightmove.co.uk/property-for-sale/...,"Added on 14/06/2021 by Griffin Property Co., W...",2022-07-10 18:02:16,v0021_300622,https://www.rightmove.co.uk/properties/35271294,0.4,...,0,0,0,0,0,0,0,0,0,0
35429088,https://www.rightmove.co.uk/properties/3542908...,599950,v0021_300622,NaN,https://www.rightmove.co.uk/property-for-sale/...,Added on 14/07/2015 by Kinleigh Folkard & Hayw...,2022-08-16 16:23:41,v0021_300622,https://www.rightmove.co.uk/properties/35429088,0.2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126179672,https://www.rightmove.co.uk/properties/1261796...,600000,v0021_300622,NaN,https://www.rightmove.co.uk/property-for-sale/...,"Added today by Bairstow Eves, Croydon South",2022-08-18 21:38:21,v0021_300622,https://www.rightmove.co.uk/properties/126179672,0.5,...,0,0,0,0,0,0,0,0,0,0
126180107,https://www.rightmove.co.uk/properties/1261801...,419999,v0021_300622,NaN,https://www.rightmove.co.uk/property-for-sale/...,"Added today by Ellis & Co, Bethnal Green",2022-08-18 21:55:19,v0021_300622,https://www.rightmove.co.uk/properties/126180107,0.2,...,0,0,0,0,0,0,0,0,0,0
126180704,https://www.rightmove.co.uk/properties/1261807...,475000,v0021_300622,NaN,https://www.rightmove.co.uk/property-for-sale/...,"Added today by Keatons, Stratford",2022-08-18 21:46:56,v0021_300622,https://www.rightmove.co.uk/properties/126180704,0.3,...,0,0,0,0,0,0,0,0,0,0
126180962,https://www.rightmove.co.uk/properties/1261809...,450000,v0021_300622,NaN,https://www.rightmove.co.uk/property-for-sale/...,"Added today by Foxtons, Walthamstow",2022-08-18 21:34:45,v0021_300622,https://www.rightmove.co.uk/properties/126180962,0.5,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#print(df.keyFeatures.str.len().describe())
#df.keyFeatures.str.lower().str.contains('garden')
keyword = 'garden'
print(df_prep.keyFeatures.str.lower().str.contains(keyword).sum())
df_prep.keyFeatures.str.lower().str.contains(keyword)
df_prep.keyFeatures.str.lower().str.index("]")

20285


14520525     208
27953107     206
33593487     222
35271294     141
35429088       1
            ... 
126179672    274
126180107    183
126180704    350
126180962    165
126181118    170
Name: keyFeatures, Length: 54067, dtype: int64

In [11]:
dailmail = ['garden','central heating','parking','off road','shower','cavity wall insulation','wall insulation','insulation','insulat','dining room','garage','en-suite','en suite']
common_knowledge = ['penthouse','balcony']
ideal_home = ['double-glazing','double glazing','off-road parking','security','patio','underfloor heating','marble']
discarded = ['signal','secure doors','secure door','outdoor lighting','bathtub','neighbours',]

#keywords = ['']
keywords = []
keywords.extend(dailmail)
keywords.extend(common_knowledge)
keywords.extend(ideal_home)
for keyword in keywords:
    print(keyword, df_prep.keyFeatures.str.lower().str.contains(keyword).sum())

garden 20285
central heating 3475
parking 15958
off road 530
shower 2901
cavity wall insulation 3
wall insulation 5
insulation 14
insulat 38
dining room 1684
garage 4645
en-suite 1403
en suite 675
penthouse 172
balcony 8672
double-glazing 22
double glazing 2502
off-road parking 27
security 495
patio 1030
underfloor heating 721
marble 56


In [12]:
#df = feature_engineer(df, version=VERSION)

In [13]:
import re
spice_df = pd.DataFrame(dict((spice, df_prep.keyFeatures.str.contains(spice, re.IGNORECASE))
                             for spice in keywords))
spice_df.head()

,garden,central heating,parking,off road,shower,cavity wall insulation,wall insulation,insulation,insulat,dining room,...,en suite,penthouse,balcony,double-glazing,double glazing,off-road parking,security,patio,underfloor heating,marble
14520525,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
27953107,False,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
33593487,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
35271294,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,False,False
35429088,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
df = df_prep[columns].merge(spice_df, how='outer', left_index=True, right_index=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54067 entries, 14520525 to 126181118
Data columns (total 81 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Price                               54067 non-null  int64  
 1   bedrooms                            52265 non-null  float64
 2   bathrooms                           50569 non-null  float64
 3   nearestStation                      54067 non-null  float64
 4   location.latitude                   54057 non-null  float64
 5   location.longitude                  54067 non-null  float64
 6   latitude_deviation                  54057 non-null  float64
 7   longitude_deviation                 54067 non-null  float64
 8   tenure.tenureType                   50413 non-null  object 
 9   feature__1 bedroom                  54067 non-null  int64  
 10  feature__2 bedrooms                 54067 non-null  int64  
 11  feature__2 double bedrooms         

In [15]:
df.head(30)

,Price,bedrooms,bathrooms,nearestStation,location.latitude,location.longitude,latitude_deviation,longitude_deviation,tenure.tenureType,feature__1 bedroom,...,en suite,penthouse,balcony,double-glazing,double glazing,off-road parking,security,patio,underfloor heating,marble
14520525,550000,3.0,1.0,0.274316,51.529950,-0.207020,0.030472,0.101800,LEASEHOLD,0,...,False,False,True,False,False,False,False,False,False,False
27953107,400000,2.0,2.0,0.305845,51.549390,-0.482600,0.049912,0.377380,LEASEHOLD,0,...,False,False,True,False,False,False,False,False,False,False
33593487,579950,2.0,1.0,0.438045,51.447180,-0.338770,0.052298,0.233550,FREEHOLD,0,...,False,False,False,False,False,False,False,False,False,False
35271294,370000,2.0,1.0,0.399307,51.449568,-0.140154,0.049910,0.034934,LEASEHOLD,0,...,False,False,True,False,True,False,False,False,False,False
35429088,599950,2.0,1.0,0.238187,51.577030,-0.141230,0.077552,0.036010,NaN,0,...,False,False,False,False,False,False,False,False,False,False
44749111,475000,2.0,1.0,0.410550,51.370050,-0.212410,0.129428,0.107190,FREEHOLD,0,...,False,False,False,False,True,False,False,False,False,False
46204665,435000,3.0,2.0,0.314779,51.539070,-0.198935,0.039592,0.093715,LEASEHOLD,0,...,True,False,False,False,False,False,False,False,False,False
49020666,200000,1.0,1.0,0.875911,51.539959,-0.380863,0.040481,0.275643,LEASEHOLD,0,...,False,False,False,False,False,False,False,False,False,False
49036279,275000,2.0,1.0,0.474368,51.541780,0.037890,0.042302,0.143110,LEASEHOLD,0,...,False,False,False,False,False,False,False,False,False,False
49303873,450000,3.0,2.0,0.577040,51.524880,0.187200,0.025402,0.292420,FREEHOLD,0,...,False,False,False,False,False,False,False,False,False,False


In [16]:
print(len(df_orig))
old_length = len(df_orig)

print(f"{old_length} ==> {len(df)}")
old_length = len(df)

54067
54067 ==> 54067


In [17]:
df['tenure.tenureType'] = df['tenure.tenureType'].astype('category')

print(f"{old_length} ==> {len(df)}")
old_length = len(df)

54067 ==> 54067


In [18]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,54067.0,416448.380528,113505.624206,100000.000000,325000.000000,425000.000000,500000.000000,600000.000000
bedrooms,52265.0,11.360145,2143.329175,1.000000,1.000000,2.000000,2.000000,490000.000000
bathrooms,50569.0,1.181534,0.413244,1.000000,1.000000,1.000000,1.000000,12.000000
nearestStation,54067.0,0.442075,1.049040,0.000000,0.221778,0.361208,0.553963,192.431869
location.latitude,54057.0,51.496130,0.076650,51.298317,51.439070,51.499478,51.554067,51.683185
location.longitude,54067.0,-0.106239,0.718769,-0.498315,-0.212965,-0.105220,-0.012998,51.558746
latitude_deviation,54057.0,0.063545,0.042994,0.000000,0.027538,0.057198,0.093844,0.201161
longitude_deviation,54067.0,0.133758,0.706214,0.000000,0.044160,0.098877,0.192164,51.663966
feature__1 bedroom,54067.0,0.010450,0.101691,0.000000,0.000000,0.000000,0.000000,1.000000
feature__2 bedrooms,54067.0,0.014593,0.120072,0.000000,0.000000,0.000000,0.000000,2.000000


In [19]:
df.describe(include=['object', 'category'], exclude=[]).T

,count,unique,top,freq
tenure.tenureType,50413,4,LEASEHOLD,31582


In [ ]:
%matplotlib inline
df.hist(bins=50, figsize=(20, 20))
plt.show()

<div id = 3 style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#b08888;
           font-size:100%;
           font-family:Verdana;
           letter-spacing:0.5px;
           display:flex;
            justify-content:center;">

<h2 style="padding: 2rem;
              color:white;
          text-align:center;
          margin:0 auto;
          ">
    IDENTIFY OUTLIERS


</h2>
</div>

In [ ]:
df.isna().sum()

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['bedrooms'] > 5].sort_values('bedrooms', ascending=False)

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['bedrooms'] > 10].sort_values('bedrooms', ascending=False)

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['bathrooms'] > 5]

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['longitude_deviation'] > 1]

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['latitude_deviation'] > 1]

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['nearestStation'] > 20]

In [ ]:
# Identifying outliers, in order to try and verify (from the source data) that they are incorrect data.
df[df['nearestStation'] > 4].sort_values('nearestStation', ascending=False)

In [ ]:
df = df[(df['bedrooms'] <= 10)]
df = df[df['bathrooms'] <= 5]
df = df[((df['longitude_deviation'] <= 1) & (df['latitude_deviation'] <= 1))]
df = df[(df['nearestStation'] <= 4)]

print(f"{old_length} ==> {len(df)}")
old_length = len(df)

df.describe().T

In [ ]:
df.hist(bins=50, figsize=(20, 20))
plt.show()

In [ ]:
#sns.pairplot(df, diag_kind='kde')

In [ ]:
#df.isna().any()
df.isna().sum()

In [ ]:
df = df.dropna()
print(f"{old_length} ==> {len(df)}")
old_length = len(df)
df.describe()

In [ ]:
df_features = df.drop(LABEL, axis=1)
df_features

In [ ]:
use_one_hot_encoding_not_dummies = False

if use_one_hot_encoding_not_dummies:
    raise NotImplementedError('not implemented one hot encoding yet')

    #from sklearn.preprocessing import OneHotEncoder
    #tenure_encoder = OneHotEncoder()
    #df_tenure_type = tenure_encoder.fit_transform(df_features[['tenure.tenureType']])
else:
    for column in categories:
        df_features = pd.concat([df_features, pd.get_dummies(df_features[column], prefix=column, drop_first=True)],
                                axis=1)
        df_features.drop([column], axis=1, inplace=True)  # now drop the original column (you don't need it anymore),

features_corr = df_features.corr()
features_corr

In [ ]:
sns.set(font_scale=5)
#plt.figure(figsize=(50, 35))
plt.figure(figsize=(80, 80))
ax = sns.heatmap(features_corr, annot=True, fmt='.1f', cmap='coolwarm')

In [ ]:
data_corr = df.corr()
data_corr_dict = data_corr["Price"].sort_values(ascending=False).to_dict()
data_corr_dict

In [ ]:
#print the most positively correlated features
pos_corr = {k: v for k, v in data_corr_dict.items() if v > 0.1 and v != 1}
pos_corr

In [ ]:
#print the most negatively correlated features
neg_corr = {k: v for k, v in data_corr_dict.items() if v < -0.1}
neg_corr

In [ ]:
plt.figure(figsize=(12, 12))
sns.set(font_scale=1.5)
sns.scatterplot(x='bedrooms', y='Price', data=df, hue='tenure.tenureType')
plt.xticks(weight='bold', rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
sns.set(font_scale=1.5)
sns.scatterplot(x='latitude_deviation', y='Price', data=df, hue='tenure.tenureType')
plt.xticks(weight='bold', rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
sns.set(font_scale=1.5)
sns.scatterplot(x='longitude_deviation', y='Price', data=df, hue='tenure.tenureType')
plt.xticks(weight='bold', rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
sns.set(font_scale=1.5)
sns.scatterplot(x='latitude_deviation', y='longitude_deviation', data=df, hue='tenure.tenureType')
plt.xticks(weight='bold', rotation=90)
plt.show()

<div id = 3 style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#b08888;
           font-size:100%;
           font-family:Verdana;
           letter-spacing:0.5px;
           display:flex;
            justify-content:center;">

<h2 style="padding: 2rem;
              color:white;
          text-align:center;
          margin:0 auto;
          ">
    Exploration for imputation


</h2>
</div>

In [ ]:
df['central_distance'] = (df['latitude_deviation'] ** 2 + df['longitude_deviation'] ** 2) ** 0.5
df

In [ ]:

plt.figure(figsize=(12, 12))
sns.set(font_scale=1.5)
sns.scatterplot(x='bedrooms', y='central_distance', data=df, hue='tenure.tenureType')
plt.xticks(weight='bold', rotation=90)
plt.show()

In [ ]:
#df.groupby(['tenure.tenureType','bedrooms']).agg({'bedrooms':['mean','min','max']})
df.groupby(['tenure.tenureType', 'bedrooms']).agg('count')